In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier


In [2]:
#read all the team ratings data
tmr11 = pd.read_csv(r'C:\Users\krist\Documents\BME\2024_2025_1\Diplomatervezes_2\data\team_ratings\tmr_2010_11.csv')
tmr12 = pd.read_csv(r'C:\Users\krist\Documents\BME\2024_2025_1\Diplomatervezes_2\data\team_ratings\tmr_2011_12.csv')
tmr13 = pd.read_csv(r'C:\Users\krist\Documents\BME\2024_2025_1\Diplomatervezes_2\data\team_ratings\tmr_2012_13.csv')
tmr14 = pd.read_csv(r'C:\Users\krist\Documents\BME\2024_2025_1\Diplomatervezes_2\data\team_ratings\tmr_2013_14.csv')
tmr15 = pd.read_csv(r'C:\Users\krist\Documents\BME\2024_2025_1\Diplomatervezes_2\data\team_ratings\tmr_2014_15.csv')
tmr16 = pd.read_csv(r'C:\Users\krist\Documents\BME\2024_2025_1\Diplomatervezes_2\data\team_ratings\tmr_2015_16.csv')
tmr17 = pd.read_csv(r'C:\Users\krist\Documents\BME\2024_2025_1\Diplomatervezes_2\data\team_ratings\tmr_2016_17.csv')
tmr18 = pd.read_csv(r'C:\Users\krist\Documents\BME\2024_2025_1\Diplomatervezes_2\data\team_ratings\tmr_2017_18.csv')
tmr19 = pd.read_csv(r'C:\Users\krist\Documents\BME\2024_2025_1\Diplomatervezes_2\data\team_ratings\tmr_2018_19.csv')
tmr20 = pd.read_csv(r'C:\Users\krist\Documents\BME\2024_2025_1\Diplomatervezes_2\data\team_ratings\tmr_2019_20.csv')
tmr21 = pd.read_csv(r'C:\Users\krist\Documents\BME\2024_2025_1\Diplomatervezes_2\data\team_ratings\tmr_2020_21.csv')
tmr22 = pd.read_csv(r'C:\Users\krist\Documents\BME\2024_2025_1\Diplomatervezes_2\data\team_ratings\tmr_2021_22.csv')
tmr23 = pd.read_csv(r'C:\Users\krist\Documents\BME\2024_2025_1\Diplomatervezes_2\data\team_ratings\tmr_2022_23.csv')

In [3]:
tmr11.head(5)

,Rk,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,1,Miami Heat,E,SE,58,24,0.707,7.46,112.62,104.35,8.27,6.76,112.43,104.94,7.49
1,2,Chicago Bulls,E,C,62,20,0.756,7.32,109.30,101.07,8.23,6.53,109.02,101.65,7.37
2,3,Los Angeles Lakers,W,P,57,25,0.695,6.11,111.61,105.00,6.61,6.01,111.40,104.89,6.51
3,4,San Antonio Spurs,W,SW,61,21,0.744,5.71,112.31,106.23,6.07,5.86,112.22,105.97,6.25
4,5,Orlando Magic,E,SE,52,30,0.634,5.46,108.78,102.87,5.91,4.92,108.63,103.31,5.32


In [5]:
#concatenate all the data but only keep the team, and the W/L% columns and add a season column
tmr11['Season'] = 2011
tmr12['Season'] = 2012
tmr13['Season'] = 2013
tmr14['Season'] = 2014
tmr15['Season'] = 2015
tmr16['Season'] = 2016
tmr17['Season'] = 2017
tmr18['Season'] = 2018
tmr19['Season'] = 2019
tmr20['Season'] = 2020
tmr21['Season'] = 2021
tmr22['Season'] = 2022
tmr23['Season'] = 2023
tmr = pd.concat([tmr11, tmr12, tmr13, tmr14, tmr15, tmr16, tmr17, tmr18, tmr19, tmr20, tmr21, tmr22], axis=0)
tmr = tmr[['Team','Rk', 'W', 'Season']] #W/L% 
tmr.head(5)

,Team,Rk,W,Season
0,Miami Heat,1,58,2011
1,Chicago Bulls,2,62,2011
2,Los Angeles Lakers,3,57,2011
3,San Antonio Spurs,4,61,2011
4,Orlando Magic,5,52,2011


In [7]:
# Add new columns for salaries
tmr['Salary'] = np.nan
tmr['Adjusted Salary'] = np.nan

# Define the salary data
salary_data = {
    'Los Angeles Lakers': (91428140, 127981772),
    'Orlando Magic': (90770804, 127061629),
    'Dallas Mavericks': (85409890, 119557381),
    'Boston Celtics': (82045867, 114848398),
    'Utah Jazz': (75855296, 106182791),
    'Portland Trail Blazers': (74584058, 104403301),
    'Houston Rockets': (72315654, 101227974),
    'Atlanta Hawks': (71469843, 100044000),
    'Memphis Grizzlies': (70241344, 98324338),
    'Toronto Raptors': (69992648, 97976210),
    'San Antonio Spurs': (69522311, 97317829),
    'Milwaukee Bucks': (69076690, 96694046),
    'Philadelphia 76ers': (68915246, 96468060),
    'Golden State Warriors': (68415336, 95768280),
    'New Orleans Hornets': (68230729, 95509865),
    'Denver Nuggets': (68019290, 95213894),
    'New York Knicks': (67016562, 93810267),
    'Charlotte Bobcats': (66542696, 93146945),
    'Miami Heat': (66205855, 92675428),
    'Phoenix Suns': (65979054, 92357954),
    'Detroit Pistons': (65517821, 91712317),
    'Indiana Pacers': (65088630, 91111532),
    'New Jersey Nets': (58495989, 81883106),
    'Washington Wizards': (58324988, 81643736),
    'Oklahoma City Thunder': (55922488, 78280698),
    'Chicago Bulls': (54538076, 76342791),
    'Cleveland Cavaliers': (53990021, 75575618),
    'Minnesota Timberwolves': (53461123, 74835262),
    'Los Angeles Clippers': (53225329, 74505195),
    'Sacramento Kings': (45268465, 63367121)
}

# Update the dataframe with the salary data for the year 2011
for team, (salary, adj_salary) in salary_data.items():
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2011), 'Salary'] = salary
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2011), 'Adjusted Salary'] = adj_salary

tmr.head(35)

,Team,Rk,W,Season,Salary,Adjusted Salary
0,Miami Heat,1,58,2011,66205855.0,92675428.0
1,Chicago Bulls,2,62,2011,54538076.0,76342791.0
2,Los Angeles Lakers,3,57,2011,91428140.0,127981772.0
3,San Antonio Spurs,4,61,2011,69522311.0,97317829.0
4,Orlando Magic,5,52,2011,90770804.0,127061629.0
5,Boston Celtics,6,56,2011,82045867.0,114848398.0
6,Denver Nuggets,7,50,2011,68019290.0,95213894.0
7,Dallas Mavericks,8,57,2011,85409890.0,119557381.0
8,Oklahoma City Thunder,9,55,2011,55922488.0,78280698.0
9,Memphis Grizzlies,10,46,2011,70241344.0,98324338.0


In [8]:
# Define the salary data for 2012
salary_data_2012 = {
    'Los Angeles Lakers': (88264478, 119307310),
    'Orlando Magic': (87955505, 118889670),
    'Portland Trail Blazers': (80920717, 109380730),
    'Phoenix Suns': (80640576, 109002063),
    'Boston Celtics': (79820530, 107893605),
    'Dallas Mavericks': (77180233, 104324707),
    'New York Knicks': (77093212, 104207081),
    'Miami Heat': (75469639, 102012493),
    'San Antonio Spurs': (73732762, 99664749),
    'Atlanta Hawks': (73669912, 99579799),
    'Detroit Pistons': (70923605, 95867610),
    'Memphis Grizzlies': (70628007, 95468049),
    'Chicago Bulls': (69548447, 94008804),
    'Los Angeles Clippers': (67333389, 91014704),
    'Philadelphia 76ers': (66672368, 90121200),
    'New Orleans Hornets': (65441008, 88456769),
    'New Jersey Nets': (65281235, 88240802),
    'Cleveland Cavaliers': (64875599, 87692501),
    'Golden State Warriors': (61576901, 83233644),
    'Milwaukee Bucks': (60192388, 81362195),
    'Oklahoma City Thunder': (59501709, 80428603),
    'Denver Nuggets': (58553173, 79146465),
    'Utah Jazz': (58178016, 78639365),
    'Charlotte Bobcats': (57902024, 78266303),
    'Houston Rockets': (57021755, 77076442),
    'Minnesota Timberwolves': (56844339, 76836631),
    'Washington Wizards': (55386345, 74865855),
    'Toronto Raptors': (55063049, 74428856),
    'Indiana Pacers': (51554083, 69685777),
    'Sacramento Kings': (48950236, 66166150)
}

# Update the dataframe with the salary data for the year 2012
for team, (salary, adj_salary) in salary_data_2012.items():
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2012), 'Salary'] = salary
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2012), 'Adjusted Salary'] = adj_salary

tmr.head(35)

,Team,Rk,W,Season,Salary,Adjusted Salary
0,Miami Heat,1,58,2011,66205855.0,92675428.0
1,Chicago Bulls,2,62,2011,54538076.0,76342791.0
2,Los Angeles Lakers,3,57,2011,91428140.0,127981772.0
3,San Antonio Spurs,4,61,2011,69522311.0,97317829.0
4,Orlando Magic,5,52,2011,90770804.0,127061629.0
5,Boston Celtics,6,56,2011,82045867.0,114848398.0
6,Denver Nuggets,7,50,2011,68019290.0,95213894.0
7,Dallas Mavericks,8,57,2011,85409890.0,119557381.0
8,Oklahoma City Thunder,9,55,2011,55922488.0,78280698.0
9,Memphis Grizzlies,10,46,2011,70241344.0,98324338.0


In [9]:
# Define the salary data for 2013
salary_data_2013 = {
    'Los Angeles Lakers': (100131988, 133133324),
    'Brooklyn Nets': (87644649, 116530425),
    'Miami Heat': (80427933, 106935238),
    'New York Knicks': (79977119, 106335843),
    'Boston Celtics': (75508399, 100394328),
    'Chicago Bulls': (74491280, 99041989),
    'Golden State Warriors': (70036985, 93119656),
    'San Antonio Spurs': (69838600, 92855888),
    'Dallas Mavericks': (69766571, 92760115),
    'Los Angeles Clippers': (69152721, 91943960),
    'Oklahoma City Thunder': (68924100, 91639991),
    'Detroit Pistons': (68807497, 91484957),
    'Atlanta Hawks': (66710178, 88696405),
    'Philadelphia 76ers': (66697839, 88680001),
    'Orlando Magic': (66557496, 88493401),
    'Indiana Pacers': (66107195, 87894693),
    'Utah Jazz': (65657569, 87296878),
    'Minnesota Timberwolves': (64329748, 85531436),
    'New Orleans Hornets': (64134537, 85271889),
    'Toronto Raptors': (63708138, 84704957),
    'Denver Nuggets': (63444254, 84354104),
    'Memphis Grizzlies': (63054115, 83835386),
    'Milwaukee Bucks': (62410041, 82979036),
    'Washington Wizards': (59715539, 79396482),
    'Portland Trail Blazers': (59442596, 79033586),
    'Charlotte Bobcats': (58122566, 77278505),
    'Cleveland Cavaliers': (54472190, 72425037),
    'Sacramento Kings': (54421935, 72358222),
    'Phoenix Suns': (54196264, 72058174),
    'Houston Rockets': (48048265, 63883927)
}

# Update the dataframe with the salary data for the year 2013
for team, (salary, adj_salary) in salary_data_2013.items():
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2013), 'Salary'] = salary
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2013), 'Adjusted Salary'] = adj_salary

tmr.head(35)

,Team,Rk,W,Season,Salary,Adjusted Salary
0,Miami Heat,1,58,2011,66205855.0,92675428.0
1,Chicago Bulls,2,62,2011,54538076.0,76342791.0
2,Los Angeles Lakers,3,57,2011,91428140.0,127981772.0
3,San Antonio Spurs,4,61,2011,69522311.0,97317829.0
4,Orlando Magic,5,52,2011,90770804.0,127061629.0
5,Boston Celtics,6,56,2011,82045867.0,114848398.0
6,Denver Nuggets,7,50,2011,68019290.0,95213894.0
7,Dallas Mavericks,8,57,2011,85409890.0,119557381.0
8,Oklahoma City Thunder,9,55,2011,55922488.0,78280698.0
9,Memphis Grizzlies,10,46,2011,70241344.0,98324338.0


In [10]:
# Define the salary data for 2014
salary_data_2014 = {
    'Brooklyn Nets': (102589967, 134049610),
    'New York Knicks': (89072787, 116387329),
    'Miami Heat': (80841617, 105632035),
    'Los Angeles Lakers': (77423614, 101165887),
    'Dallas Mavericks': (75012158, 98014947),
    'Chicago Bulls': (73336893, 95825953),
    'Los Angeles Clippers': (73049774, 95450790),
    'Golden State Warriors': (72503122, 94736505),
    'Memphis Grizzlies': (71992885, 94069802),
    'Oklahoma City Thunder': (71653401, 93626214),
    'Toronto Raptors': (71561643, 93506317),
    'Boston Celtics': (70885503, 92622838),
    'Minnesota Timberwolves': (68564040, 89589487),
    'New Orleans Pelicans': (67358651, 88014459),
    'Indiana Pacers': (67182414, 87784181),
    'Cleveland Cavaliers': (66654061, 87093805),
    'Denver Nuggets': (65782065, 85954408),
    'Washington Wizards': (64698822, 84538984),
    'San Antonio Spurs': (63115911, 82470664),
    'Detroit Pistons': (62387432, 81518796),
    'Portland Trail Blazers': (61260350, 80046090),
    'Charlotte Bobcats': (60395651, 78916229),
    'Sacramento Kings': (59174155, 77320154),
    'Atlanta Hawks': (58998677, 77090865),
    'Houston Rockets': (57823063, 75554742),
    'Orlando Magic': (56948031, 74411377),
    'Utah Jazz': (56911999, 74364299),
    'Phoenix Suns': (55569468, 72610074),
    'Milwaukee Bucks': (54400846, 71083089),
    'Philadelphia 76ers': (51741312, 67607994)
}

# Update the dataframe with the salary data for the year 2014
for team, (salary, adj_salary) in salary_data_2014.items():
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2014), 'Salary'] = salary
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2014), 'Adjusted Salary'] = adj_salary

tmr.head(35)

,Team,Rk,W,Season,Salary,Adjusted Salary
0,Miami Heat,1,58,2011,66205855.0,92675428.0
1,Chicago Bulls,2,62,2011,54538076.0,76342791.0
2,Los Angeles Lakers,3,57,2011,91428140.0,127981772.0
3,San Antonio Spurs,4,61,2011,69522311.0,97317829.0
4,Orlando Magic,5,52,2011,90770804.0,127061629.0
5,Boston Celtics,6,56,2011,82045867.0,114848398.0
6,Denver Nuggets,7,50,2011,68019290.0,95213894.0
7,Dallas Mavericks,8,57,2011,85409890.0,119557381.0
8,Oklahoma City Thunder,9,55,2011,55922488.0,78280698.0
9,Memphis Grizzlies,10,46,2011,70241344.0,98324338.0


In [11]:
# Define the salary data for 2015
salary_data_2015 = {
    'Brooklyn Nets': (87817289, 112417160),
    'Los Angeles Clippers': (85259447, 109142799),
    'Dallas Mavericks': (84820225, 108580541),
    'Cleveland Cavaliers': (82592827, 105729192),
    'New York Knicks': (81369219, 104162818),
    'Toronto Raptors': (81317963, 104097208),
    'Oklahoma City Thunder': (80408520, 102933006),
    'New Orleans Pelicans': (80012866, 102426515),
    'Los Angeles Lakers': (77546500, 99269258),
    'Houston Rockets': (76849302, 98376758),
    'Miami Heat': (75813041, 97050213),
    'Memphis Grizzlies': (75158730, 96212612),
    'Indiana Pacers': (74793526, 95745106),
    'Golden State Warriors': (73611166, 94231539),
    'Sacramento Kings': (73379071, 93934425),
    'Washington Wizards': (73372974, 93926623),
    'San Antonio Spurs': (70770209, 90594752),
    'Portland Trail Blazers': (69936842, 89527940),
    'Minnesota Timberwolves': (69378954, 88813769),
    'Chicago Bulls': (67414682, 86299261),
    'Detroit Pistons': (67410675, 86294129),
    'Charlotte Hornets': (66792937, 85503349),
    'Boston Celtics': (62218516, 79647509),
    'Utah Jazz': (60847538, 77892486),
    'Orlando Magic': (59629210, 76332875),
    'Atlanta Hawks': (58320278, 74657278),
    'Phoenix Suns': (58018672, 74271184),
    'Denver Nuggets': (56820307, 72737126),
    'Milwaukee Bucks': (54473275, 69732630),
    'Philadelphia 76ers': (54355571, 69581950)
}

# Update the dataframe with the salary data for the year 2015
for team, (salary, adj_salary) in salary_data_2015.items():
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2015), 'Salary'] = salary
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2015), 'Adjusted Salary'] = adj_salary

tmr.head(35)

,Team,Rk,W,Season,Salary,Adjusted Salary
0,Miami Heat,1,58,2011,66205855.0,92675428.0
1,Chicago Bulls,2,62,2011,54538076.0,76342791.0
2,Los Angeles Lakers,3,57,2011,91428140.0,127981772.0
3,San Antonio Spurs,4,61,2011,69522311.0,97317829.0
4,Orlando Magic,5,52,2011,90770804.0,127061629.0
5,Boston Celtics,6,56,2011,82045867.0,114848398.0
6,Denver Nuggets,7,50,2011,68019290.0,95213894.0
7,Dallas Mavericks,8,57,2011,85409890.0,119557381.0
8,Oklahoma City Thunder,9,55,2011,55922488.0,78280698.0
9,Memphis Grizzlies,10,46,2011,70241344.0,98324338.0


In [12]:
# Define the salary data for 2016
salary_data_2016 = {
    'Cleveland Cavaliers': (108300458, 138466811),
    'Los Angeles Clippers': (97019321, 124043388),
    'Oklahoma City Thunder': (95708387, 122367304),
    'Golden State Warriors': (93669566, 119760582),
    'San Antonio Spurs': (87832839, 112298074),
    'Houston Rockets': (87504058, 111877712),
    'Chicago Bulls': (87073838, 111327660),
    'Miami Heat': (85764781, 109653972),
    'Washington Wizards': (85055155, 108746681),
    'New Orleans Pelicans': (83709371, 107026041),
    'Memphis Grizzlies': (83223881, 106405320),
    'Brooklyn Nets': (80258302, 102613698),
    'Detroit Pistons': (77256014, 98775145),
    'Boston Celtics': (77141919, 98629268),
    'Charlotte Hornets': (76860006, 98268833),
    'Dallas Mavericks': (75397067, 96398402),
    'New York Knicks': (74237021, 94915233),
    'Milwaukee Bucks': (73843541, 94412153),
    'Los Angeles Lakers': (72694352, 92942865),
    'Denver Nuggets': (72589023, 92808194),
    'Minnesota Timberwolves': (72287243, 92422357),
    'Atlanta Hawks': (71661760, 91622652),
    'Indiana Pacers': (71605233, 91550376),
    'Toronto Raptors': (71591189, 91532422),
    'Sacramento Kings': (70610560, 90278644),
    'Phoenix Suns': (68095365, 87062858),
    'Philadelphia 76ers': (64583220, 82572427),
    'Utah Jazz': (63608425, 81326111),
    'Orlando Magic': (63199651, 80803478),
    'Portland Trail Blazers': (61685814, 78867974)
}

# Update the dataframe with the salary data for the year 2016
for team, (salary, adj_salary) in salary_data_2016.items():
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2016), 'Salary'] = salary
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2016), 'Adjusted Salary'] = adj_salary

tmr.head(35)

,Team,Rk,W,Season,Salary,Adjusted Salary
0,Miami Heat,1,58,2011,66205855.0,92675428.0
1,Chicago Bulls,2,62,2011,54538076.0,76342791.0
2,Los Angeles Lakers,3,57,2011,91428140.0,127981772.0
3,San Antonio Spurs,4,61,2011,69522311.0,97317829.0
4,Orlando Magic,5,52,2011,90770804.0,127061629.0
5,Boston Celtics,6,56,2011,82045867.0,114848398.0
6,Denver Nuggets,7,50,2011,68019290.0,95213894.0
7,Dallas Mavericks,8,57,2011,85409890.0,119557381.0
8,Oklahoma City Thunder,9,55,2011,55922488.0,78280698.0
9,Memphis Grizzlies,10,46,2011,70241344.0,98324338.0


In [13]:
# Define the salary data for 2017
salary_data_2017 = {
    'Cleveland Cavaliers': (128522489, 162698908),
    'Portland Trail Blazers': (119732234, 151571171),
    'Los Angeles Clippers': (114756766, 145272637),
    'San Antonio Spurs': (112017779, 141805301),
    'Memphis Grizzlies': (110083520, 139356687),
    'Toronto Raptors': (108599970, 137478636),
    'Detroit Pistons': (106492988, 134811372),
    'Orlando Magic': (104096951, 131778186),
    'Washington Wizards': (104016580, 131676439),
    'Charlotte Hornets': (103054004, 130457897),
    'New York Knicks': (102593418, 129874836),
    'Dallas Mavericks': (102354966, 129572971),
    'New Orleans Pelicans': (101616451, 128638072),
    'Golden State Warriors': (101584835, 128598048),
    'Miami Heat': (100740770, 127529534),
    'Atlanta Hawks': (96315163, 121927078),
    'Milwaukee Bucks': (96245877, 121839369),
    'Sacramento Kings': (95596327, 121017093),
    'Los Angeles Lakers': (94781848, 119986028),
    'Boston Celtics': (93465326, 118319419),
    'Chicago Bulls': (92522306, 117125635),
    'Oklahoma City Thunder': (91230089, 115489792),
    'Houston Rockets': (90956067, 115142904),
    'Indiana Pacers': (90279072, 114285882),
    'Phoenix Suns': (89754590, 113621931),
    'Philadelphia 76ers': (84775343, 107318610),
    'Denver Nuggets': (83836460, 106130061),
    'Minnesota Timberwolves': (83527580, 105739049),
    'Brooklyn Nets': (82391482, 104300840),
    'Utah Jazz': (80138192, 101448361)
}

# Update the dataframe with the salary data for the year 2017
for team, (salary, adj_salary) in salary_data_2017.items():
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2017), 'Salary'] = salary
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2017), 'Adjusted Salary'] = adj_salary

tmr.head(35)

,Team,Rk,W,Season,Salary,Adjusted Salary
0,Miami Heat,1,58,2011,66205855.0,92675428.0
1,Chicago Bulls,2,62,2011,54538076.0,76342791.0
2,Los Angeles Lakers,3,57,2011,91428140.0,127981772.0
3,San Antonio Spurs,4,61,2011,69522311.0,97317829.0
4,Orlando Magic,5,52,2011,90770804.0,127061629.0
5,Boston Celtics,6,56,2011,82045867.0,114848398.0
6,Denver Nuggets,7,50,2011,68019290.0,95213894.0
7,Dallas Mavericks,8,57,2011,85409890.0,119557381.0
8,Oklahoma City Thunder,9,55,2011,55922488.0,78280698.0
9,Memphis Grizzlies,10,46,2011,70241344.0,98324338.0


In [14]:
# Define the salary data for 2018
salary_data_2018 = {
    'Cleveland Cavaliers': (137722926, 171543761),
    'Golden State Warriors': (137610134, 171403270),
    'Oklahoma City Thunder': (134534640, 167572521),
    'Miami Heat': (133624374, 166438720),
    'Washington Wizards': (123306396, 153586941),
    'New Orleans Pelicans': (120814452, 150483043),
    'Milwaukee Bucks': (120521249, 150117838),
    'Houston Rockets': (119905532, 149350917),
    'Detroit Pistons': (119773191, 149186077),
    'Los Angeles Clippers': (119093010, 148338864),
    'Portland Trail Blazers': (118708146, 147859492),
    'Charlotte Hornets': (117382664, 146208509),
    'Toronto Raptors': (116929373, 145643903),
    'Minnesota Timberwolves': (116075131, 144579881),
    'Boston Celtics': (115284776, 143595440),
    'San Antonio Spurs': (114633844, 142784658),
    'Memphis Grizzlies': (110700149, 137884954),
    'Denver Nuggets': (107543599, 133953246),
    'Utah Jazz': (105606838, 131540873),
    'New York Knicks': (105403130, 131287140),
    'Los Angeles Lakers': (103126557, 128451503),
    'Philadelphia 76ers': (100797386, 125550355),
    'Atlanta Hawks': (99992696, 124548054),
    'Sacramento Kings': (99587185, 124042963),
    'Orlando Magic': (98700258, 122938234),
    'Brooklyn Nets': (95475397, 118921438),
    'Indiana Pacers': (95271736, 118667761),
    'Phoenix Suns': (92684083, 115444654),
    'Chicago Bulls': (90466801, 112682869),
    'Dallas Mavericks': (85440245, 106421937)
}

# Update the dataframe with the salary data for the year 2018
for team, (salary, adj_salary) in salary_data_2018.items():
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2018), 'Salary'] = salary
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2018), 'Adjusted Salary'] = adj_salary

tmr.head(35)

,Team,Rk,W,Season,Salary,Adjusted Salary
0,Miami Heat,1,58,2011,66205855.0,92675428.0
1,Chicago Bulls,2,62,2011,54538076.0,76342791.0
2,Los Angeles Lakers,3,57,2011,91428140.0,127981772.0
3,San Antonio Spurs,4,61,2011,69522311.0,97317829.0
4,Orlando Magic,5,52,2011,90770804.0,127061629.0
5,Boston Celtics,6,56,2011,82045867.0,114848398.0
6,Denver Nuggets,7,50,2011,68019290.0,95213894.0
7,Dallas Mavericks,8,57,2011,85409890.0,119557381.0
8,Oklahoma City Thunder,9,55,2011,55922488.0,78280698.0
9,Memphis Grizzlies,10,46,2011,70241344.0,98324338.0


In [15]:
# Define the salary data for 2019
salary_data_2019 = {
    'Miami Heat': (153171497, 185460477),
    'Golden State Warriors': (146291276, 177129886),
    'Oklahoma City Thunder': (144916427, 175465213),
    'Toronto Raptors': (137793831, 166841157),
    'Milwaukee Bucks': (130988604, 158601366),
    'Portland Trail Blazers': (130256600, 157715055),
    'Detroit Pistons': (126557932, 153236694),
    'Houston Rockets': (126474100, 153135190),
    'Boston Celtics': (125334993, 151755957),
    'Memphis Grizzlies': (125188633, 151578742),
    'Washington Wizards': (123747588, 149833921),
    'New York Knicks': (123387454, 149397869),
    'Cleveland Cavaliers': (123255073, 149237583),
    'Minnesota Timberwolves': (121962221, 147672196),
    'San Antonio Spurs': (121588790, 147220044),
    'Charlotte Hornets': (121427859, 147025190),
    'Brooklyn Nets': (118850600, 143904634),
    'Denver Nuggets': (118327016, 143270678),
    'Los Angeles Clippers': (118026816, 142907193),
    'New Orleans Pelicans': (116052756, 140516999),
    'Philadelphia 76ers': (115127167, 139396292),
    'Orlando Magic': (114394213, 138508829),
    'Utah Jazz': (113826156, 137821025),
    'Chicago Bulls': (112598201, 136334209),
    'Indiana Pacers': (110724804, 134065894),
    'Phoenix Suns': (108692835, 131605581),
    'Los Angeles Lakers': (107225482, 129828909),
    'Sacramento Kings': (101466920, 122856426),
    'Dallas Mavericks': (86958881, 105290052),
    'Atlanta Hawks': (79180081, 95871454)
}

# Update the dataframe with the salary data for the year 2019
for team, (salary, adj_salary) in salary_data_2019.items():
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2019), 'Salary'] = salary
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2019), 'Adjusted Salary'] = adj_salary

tmr.head(35)

,Team,Rk,W,Season,Salary,Adjusted Salary
0,Miami Heat,1,58,2011,66205855.0,92675428.0
1,Chicago Bulls,2,62,2011,54538076.0,76342791.0
2,Los Angeles Lakers,3,57,2011,91428140.0,127981772.0
3,San Antonio Spurs,4,61,2011,69522311.0,97317829.0
4,Orlando Magic,5,52,2011,90770804.0,127061629.0
5,Boston Celtics,6,56,2011,82045867.0,114848398.0
6,Denver Nuggets,7,50,2011,68019290.0,95213894.0
7,Dallas Mavericks,8,57,2011,85409890.0,119557381.0
8,Oklahoma City Thunder,9,55,2011,55922488.0,78280698.0
9,Memphis Grizzlies,10,46,2011,70241344.0,98324338.0


In [16]:
# Define the salary data for 2020
salary_data_2020 = {
    'Oklahoma City Thunder': (132017938, 157255357),
    'Portland Trail Blazers': (131979953, 157210110),
    'Los Angeles Clippers': (131506341, 156645959),
    'Cleveland Cavaliers': (131059022, 156113126),
    'Philadelphia 76ers': (129912339, 154747238),
    'Miami Heat': (129867871, 154694268),
    'Golden State Warriors': (129254928, 153964152),
    'Denver Nuggets': (128746180, 153358147),
    'Houston Rockets': (128109922, 152600258),
    'Orlando Magic': (126095610, 150200877),
    'Los Angeles Lakers': (123971686, 147670928),
    'Milwaukee Bucks': (122612183, 146051535),
    'Toronto Raptors': (122463495, 145874420),
    'Washington Wizards': (121296256, 144484046),
    'Dallas Mavericks': (120871082, 143977595),
    'San Antonio Spurs': (119217331, 142007698),
    'Utah Jazz': (118910311, 141641988),
    'Brooklyn Nets': (118889943, 141617727),
    'New Orleans Pelicans': (117868297, 140400776),
    'Boston Celtics': (117759332, 140270982),
    'Minnesota Timberwolves': (114202982, 136034777),
    'Sacramento Kings': (113796966, 135551141),
    'Indiana Pacers': (112872260, 134449664),
    'Chicago Bulls': (112601901, 134127618),
    'Atlanta Hawks': (110702618, 131865260),
    'Detroit Pistons': (104527576, 124509753),
    'New York Knicks': (100232129, 119393160),
    'Phoenix Suns': (98539675, 117377165),
    'Memphis Grizzlies': (98495848, 117324961),
    'Charlotte Hornets': (96552033, 115009553)
}

# Update the dataframe with the salary data for the year 2020
for team, (salary, adj_salary) in salary_data_2020.items():
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2020), 'Salary'] = salary
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2020), 'Adjusted Salary'] = adj_salary

tmr.head(35)

,Team,Rk,W,Season,Salary,Adjusted Salary
0,Miami Heat,1,58,2011,66205855.0,92675428.0
1,Chicago Bulls,2,62,2011,54538076.0,76342791.0
2,Los Angeles Lakers,3,57,2011,91428140.0,127981772.0
3,San Antonio Spurs,4,61,2011,69522311.0,97317829.0
4,Orlando Magic,5,52,2011,90770804.0,127061629.0
5,Boston Celtics,6,56,2011,82045867.0,114848398.0
6,Denver Nuggets,7,50,2011,68019290.0,95213894.0
7,Dallas Mavericks,8,57,2011,85409890.0,119557381.0
8,Oklahoma City Thunder,9,55,2011,55922488.0,78280698.0
9,Memphis Grizzlies,10,46,2011,70241344.0,98324338.0


In [17]:
# Define the salary data for 2021
salary_data_2021 = {
    'Golden State Warriors': (171105334, 202507304),
    'Brooklyn Nets': (170444633, 201725345),
    'Philadelphia 76ers': (147825311, 174954830),
    'Los Angeles Clippers': (139722606, 165365082),
    'Los Angeles Lakers': (139334713, 164906001),
    'Utah Jazz': (136881324, 162002356),
    'Milwaukee Bucks': (136623929, 161697720),
    'Miami Heat': (134731235, 159457673),
    'New Orleans Pelicans': (133901495, 158475655),
    'Boston Celtics': (132931565, 157327721),
    'Memphis Grizzlies': (132022601, 156251937),
    'Portland Trail Blazers': (131904647, 156112338),
    'Houston Rockets': (131784255, 155969849),
    'Washington Wizards': (131294012, 155389636),
    'Minnesota Timberwolves': (130334934, 154254547),
    'Indiana Pacers': (130237102, 154138758),
    'Denver Nuggets': (129793210, 153613401),
    'Cleveland Cavaliers': (129605319, 153391029),
    'San Antonio Spurs': (129537825, 153311146),
    'Toronto Raptors': (129131910, 152830736),
    'Chicago Bulls': (128963580, 152631513),
    'Phoenix Suns': (128858241, 152506844),
    'Dallas Mavericks': (127657823, 151086118),
    'Orlando Magic': (121739163, 144081242),
    'Atlanta Hawks': (118804016, 140607425),
    'Detroit Pistons': (117041599, 138521558),
    'Charlotte Hornets': (108218809, 128079577),
    'Sacramento Kings': (106847430, 126456514),
    'New York Knicks': (102137151, 120881782),
    'Oklahoma City Thunder': (95774839, 113351829)
}

# Update the dataframe with the salary data for the year 2021
for team, (salary, adj_salary) in salary_data_2021.items():
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2021), 'Salary'] = salary
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2021), 'Adjusted Salary'] = adj_salary

tmr.head(35)

,Team,Rk,W,Season,Salary,Adjusted Salary
0,Miami Heat,1,58,2011,66205855.0,92675428.0
1,Chicago Bulls,2,62,2011,54538076.0,76342791.0
2,Los Angeles Lakers,3,57,2011,91428140.0,127981772.0
3,San Antonio Spurs,4,61,2011,69522311.0,97317829.0
4,Orlando Magic,5,52,2011,90770804.0,127061629.0
5,Boston Celtics,6,56,2011,82045867.0,114848398.0
6,Denver Nuggets,7,50,2011,68019290.0,95213894.0
7,Dallas Mavericks,8,57,2011,85409890.0,119557381.0
8,Oklahoma City Thunder,9,55,2011,55922488.0,78280698.0
9,Memphis Grizzlies,10,46,2011,70241344.0,98324338.0


In [18]:
# Define the salary data for 2022
salary_data_2022 = {
    'Golden State Warriors': (178980766, 200991697),
    'Brooklyn Nets': (174811922, 196310168),
    'Los Angeles Clippers': (168378382, 189085437),
    'Los Angeles Lakers': (164409293, 184628228),
    'Milwaukee Bucks': (160875421, 180659762),
    'Utah Jazz': (149760719, 168178186),
    'Philadelphia 76ers': (148922969, 167237408),
    'Miami Heat': (140840240, 158160668),
    'Indiana Pacers': (138181486, 155174941),
    'Denver Nuggets': (137963926, 154930630),
    'Minnesota Timberwolves': (137432702, 154334075),
    'Boston Celtics': (136557646, 153351404),
    'Phoenix Suns': (136476474, 153260253),
    'Cleveland Cavaliers': (136385911, 153158551),
    'Chicago Bulls': (136083814, 152819302),
    'New Orleans Pelicans': (135793968, 152493812),
    'Atlanta Hawks': (135166020, 151788639),
    'Toronto Raptors': (134896484, 151485952),
    'Houston Rockets': (132267085, 148533194),
    'Detroit Pistons': (131120355, 147245439),
    'Sacramento Kings': (130457848, 146501460),
    'Washington Wizards': (128019790, 143763568),
    'San Antonio Spurs': (127655401, 143354367),
    'Orlando Magic': (126786646, 142378773),
    'Dallas Mavericks': (126696965, 142278062),
    'Portland Trail Blazers': (124788473, 140134868),
    'Charlotte Hornets': (122139566, 137160202),
    'New York Knicks': (120644081, 135480801),
    'Memphis Grizzlies': (117284457, 131708009),
    'Oklahoma City Thunder': (82022873, 92109977)
}

# Update the dataframe with the salary data for the year 2022
for team, (salary, adj_salary) in salary_data_2022.items():
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2022), 'Salary'] = salary
    tmr.loc[(tmr['Team'] == team) & (tmr['Season'] == 2022), 'Adjusted Salary'] = adj_salary

tmr.head(35)

,Team,Rk,W,Season,Salary,Adjusted Salary
0,Miami Heat,1,58,2011,66205855.0,92675428.0
1,Chicago Bulls,2,62,2011,54538076.0,76342791.0
2,Los Angeles Lakers,3,57,2011,91428140.0,127981772.0
3,San Antonio Spurs,4,61,2011,69522311.0,97317829.0
4,Orlando Magic,5,52,2011,90770804.0,127061629.0
5,Boston Celtics,6,56,2011,82045867.0,114848398.0
6,Denver Nuggets,7,50,2011,68019290.0,95213894.0
7,Dallas Mavericks,8,57,2011,85409890.0,119557381.0
8,Oklahoma City Thunder,9,55,2011,55922488.0,78280698.0
9,Memphis Grizzlies,10,46,2011,70241344.0,98324338.0


In [19]:
#print the nan rows
tmr[tmr['Salary'].isnull()]

,Team,Rk,W,Season,Salary,Adjusted Salary


In [20]:
def label_team(row):
    if row['W'] >= 42:
        return 'Contending'
    elif row['W'] >= 29:
        return 'Rebuilding'
    else:
        return 'Tanking'

tmr['Status'] = tmr.apply(label_team, axis=1)
tmr.head(35)

,Team,Rk,W,Season,Salary,Adjusted Salary,Status
0,Miami Heat,1,58,2011,66205855.0,92675428.0,Contending
1,Chicago Bulls,2,62,2011,54538076.0,76342791.0,Contending
2,Los Angeles Lakers,3,57,2011,91428140.0,127981772.0,Contending
3,San Antonio Spurs,4,61,2011,69522311.0,97317829.0,Contending
4,Orlando Magic,5,52,2011,90770804.0,127061629.0,Contending
5,Boston Celtics,6,56,2011,82045867.0,114848398.0,Contending
6,Denver Nuggets,7,50,2011,68019290.0,95213894.0,Contending
7,Dallas Mavericks,8,57,2011,85409890.0,119557381.0,Contending
8,Oklahoma City Thunder,9,55,2011,55922488.0,78280698.0,Contending
9,Memphis Grizzlies,10,46,2011,70241344.0,98324338.0,Contending


In [21]:
tmr['Salary_Rank'] = tmr.groupby('Season')['Salary'].rank(ascending=False)

In [22]:
tmr.head(5)

,Team,Rk,W,Season,Salary,Adjusted Salary,Status,Salary_Rank
0,Miami Heat,1,58,2011,66205855.0,92675428.0,Contending,19.0
1,Chicago Bulls,2,62,2011,54538076.0,76342791.0,Contending,26.0
2,Los Angeles Lakers,3,57,2011,91428140.0,127981772.0,Contending,1.0
3,San Antonio Spurs,4,61,2011,69522311.0,97317829.0,Contending,11.0
4,Orlando Magic,5,52,2011,90770804.0,127061629.0,Contending,2.0


In [24]:
from catboost import CatBoostRegressor

# Prepare the data
X_train = tmr[[ 'Rk','Adjusted Salary','Salary_Rank', 'Status']]
y_train = tmr['W'].astype(int)

# Encode the categorical feature 'Status'
X_train['Status'] = X_train['Status'].astype('category').cat.codes

# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, verbose=0)

# Train the model
model.fit(X_train, y_train)

# Predict the W/L% for the tmr dataset
X_tmr = tmr[['Rk','Adjusted Salary', 'Salary_Rank', 'Status']]
X_tmr['Status'] = X_tmr['Status'].astype('category').cat.codes
tmr['Predicted W'] = model.predict(X_tmr)

print(tmr[['Team', 'Season', 'W', 'Predicted W']].head())

C:\Users\krist\AppData\Local\Temp\ipykernel_14188\2079880086.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Status'] = X_train['Status'].astype('category').cat.codes


                 Team  Season   W  Predicted W
0          Miami Heat    2011  58    58.289557
1       Chicago Bulls    2011  62    61.856914
2  Los Angeles Lakers    2011  57    56.886941
3   San Antonio Spurs    2011  61    60.002718
4       Orlando Magic    2011  52    52.353495


C:\Users\krist\AppData\Local\Temp\ipykernel_14188\2079880086.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_tmr['Status'] = X_tmr['Status'].astype('category').cat.codes


In [ ]:
# Get feature importance
feature_importance = model.get_feature_importance(prettified=True)
print(feature_importance)

        Feature Id  Importances
0           Status    45.168004
1               Rk    36.177984
2  Adjusted Salary    10.977433
3      Salary_Rank     7.676579


In [56]:
# Calculate the mean absolute error
mae = np.mean(np.abs(tmr['W'] - tmr['Predicted W']))
print('Mean Absolute Error:', mae)


Mean Absolute Error: 1.0920463683872303


In [26]:
# Add the contracts to the teams
contracts = [
    158.588899, 154.265858, 151.371622, 129.122065, 127.408459, 150.663686, 160.732460, 130.745582, 152.907196, 127.924446,
    155.853288, 130.070303, 157.330665, 129.187181, 149.679315, 154.591641, 154.086813, 127.693998, 152.506642, 127.697804,
    158.472909, 129.732825, 128.500753, 125.864430, 128.717067, 130.696373, 133.216224, 154.731006, 129.517431, 128.028307

]

# Define the status data
status_data = [
    'contend', 'contend', 'contend', 'contend', 'contend', 'contend', 'contend', 'contend', 'contend', 'contend',
    'contend', 'contend', 'contend', 'rebuild', 'rebuild', 'contend', 'contend', 'rebuild', 'contend', 'rebuild',
    'contend', 'rebuild', 'rebuild', 'rebuild', 'rebuild', 'rebuild', 'rebuild', 'tank', 'rebuild', 'tank'
]

# Add a new column for the statuses in tmr23
tmr23['Status'] = status_data

#a new column for the contracts in tmr23
tmr23['Adjusted Salary'] = contracts


tmr23['Salary_Rank'] = tmr23['Adjusted Salary'].rank(ascending=False)
tmr23


,Rk,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,Season,Status,Adjusted Salary,Salary_Rank
0,1,Boston Celtics,E,A,57,25,0.695,6.52,118.98,112.28,6.70,6.38,119.04,112.48,6.56,2023,contend,158.588899,2.0
1,2,Cleveland Cavaliers,E,C,51,31,0.622,5.38,117.31,111.69,5.62,5.23,117.31,111.84,5.47,2023,contend,154.265858,8.0
2,3,Philadelphia 76ers,E,A,54,28,0.659,4.32,118.87,114.51,4.35,4.37,118.98,114.56,4.42,2023,contend,151.371622,12.0
3,4,Memphis Grizzlies,W,SW,51,31,0.622,3.94,116.01,112.07,3.93,3.60,115.68,112.09,3.59,2023,contend,129.122065,22.0
4,5,Milwaukee Bucks,E,C,58,24,0.707,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,2023,contend,127.408459,29.0
5,6,New York Knicks,E,A,47,35,0.573,2.93,118.86,115.83,3.03,2.99,118.98,115.88,3.10,2023,contend,150.663686,13.0
6,7,Denver Nuggets,W,NW,53,29,0.646,3.33,118.38,115.05,3.34,3.04,118.22,115.18,3.04,2023,contend,160.732460,1.0
7,8,Sacramento Kings,W,P,48,34,0.585,2.65,119.97,117.48,2.49,2.30,119.86,117.72,2.14,2023,contend,130.745582,16.0
8,9,Phoenix Suns,W,P,45,37,0.549,2.07,116.05,113.94,2.12,2.08,115.97,113.86,2.11,2023,contend,152.907196,10.0
9,10,Toronto Raptors,E,A,41,41,0.500,1.48,116.37,114.86,1.51,1.59,116.55,114.91,1.63,2023,contend,127.924446,26.0


In [27]:
#fit the model on the tmr23 dataset
X_tmr23 = tmr23[['Rk','Adjusted Salary', 'Salary_Rank', 'Status']]
X_tmr23['Status'] = X_tmr23['Status'].astype('category').cat.codes
tmr23['Predicted W'] = model.predict(X_tmr23)

print(tmr23[['Team', 'Season', 'W', 'Predicted W']].head())

                  Team  Season   W  Predicted W
0       Boston Celtics    2023  57    56.226538
1  Cleveland Cavaliers    2023  51    57.086465
2   Philadelphia 76ers    2023  54    54.747461
3    Memphis Grizzlies    2023  51    51.643563
4      Milwaukee Bucks    2023  58    48.466373


C:\Users\krist\AppData\Local\Temp\ipykernel_14188\2100368009.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_tmr23['Status'] = X_tmr23['Status'].astype('category').cat.codes


In [28]:
# Normalize the Predicted W so that their sum is 1230
total_sum = tmr23['Predicted W'].sum()

#make sure it is an integer
tmr23['Normalized Predicted W'] = tmr23['Predicted W'] * (1230 / total_sum)

#make sure it is an integer



# Print the updated dataframe
tmr23[['Team', 'Season', 'W', 'Predicted W', 'Normalized Predicted W']].head(30)

,Team,Season,W,Predicted W,Normalized Predicted W
0,Boston Celtics,2023,57,56.226538,55.268896
1,Cleveland Cavaliers,2023,51,57.086465,56.114176
2,Philadelphia 76ers,2023,54,54.747461,53.815011
3,Memphis Grizzlies,2023,51,51.643563,50.763977
4,Milwaukee Bucks,2023,58,48.466373,47.640901
5,New York Knicks,2023,47,53.447350,52.537043
6,Denver Nuggets,2023,53,52.619386,51.723180
7,Sacramento Kings,2023,48,50.175893,49.321305
8,Phoenix Suns,2023,45,52.246902,51.357041
9,Toronto Raptors,2023,41,48.197810,47.376912


In [29]:
#sum of the normalized predicted W/L%
tmr23['Normalized Predicted W'].sum()

1230.0000000000002

# Calculate income

In [32]:
player_cl = pd.read_csv(r'C:\Users\krist\Documents\BME\2024_2025_1\Diplomatervezes_2\data\ML_outputs\player_clusters.csv')
team_cl = pd.read_csv(r'C:\Users\krist\Documents\BME\2024_2025_1\Diplomatervezes_2\data\ML_outputs\team_clusters.csv')
team_cl.head()

,Team,Rk_x,Age,W_x,L_x,PW,PL,MOV_x,SOS,SRS,...,Acquired via Expansion,Acquired via Free Agency,Total Roster,Cluster_1,Cluster_2,Cluster_3,Cluster_4,Cluster_5,Cluster_6,Cluster
0,BOS,1.0,27.4,57.0,25.0,57,25,6.52,-0.15,6.38,...,0,5,15,2,2,3,2,4,5,0
1,CLE,2.0,25.4,51.0,31.0,55,27,5.38,-0.15,5.23,...,0,7,15,1,2,1,6,2,6,0
2,PHI,3.0,28.2,54.0,28.0,52,30,4.32,0.06,4.37,...,0,7,15,2,0,3,6,3,7,0
3,MEM,4.0,24.4,51.0,31.0,51,31,3.94,-0.34,3.60,...,0,3,15,1,4,2,6,1,5,0
4,MIL,5.0,29.8,58.0,24.0,50,32,3.63,-0.02,3.61,...,0,9,15,1,2,2,5,2,9,0


In [34]:
#calculate a team star power by the number of players in the first and second cluster, with the first cluster being worth 2 points and the second cluster being worth 1 point
team_star_power_df = pd.DataFrame(None)
team_star_power_df['Team'] = team_cl['Team']
team_star_power_df['Star Power'] = (2 * team_cl['Cluster_1'] + team_cl['Cluster_2'])/2
team_star_power_df['Perf'] = tmr23['Normalized Predicted W']
team_star_power_df.head()

,Team,Star Power,Perf
0,BOS,3.0,55.268896
1,CLE,2.0,56.114176
2,PHI,2.0,53.815011
3,MEM,3.0,50.763977
4,MIL,2.0,47.640901
